In [ ]:
!pip install pycountry

     |████████████████████████████████| 10.1 MB 11.8 MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=46431848e38694416a6f867ea4ebd92d0e1f4f5fe57a5a3a6d767a7f8d73a03d
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built pycountry


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pycountry
import matplotlib.pyplot as plt
import warnings
import math
import numpy as np
import keras
import tensorflow as tf

from numpy.random import seed
seed(1)


from keras.preprocessing.sequence import TimeseriesGenerator
warnings.filterwarnings("ignore")

In [ ]:
# SDG-3-Health
url_cases = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/WHO-COVID-19-global-data.csv'
url_table = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/WHO-COVID-19-global-table-data.csv'
url_symptoms = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/covid-19_symptoms.txt'
url_lawatlas = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/lawatlas-emergency-declarations.csv'
url_locations = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/locations.txt'
ulr_owid = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/owid-covid-data.csv'
url_vaccination = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/vaccination-data.csv'
url_US = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-3-Health/lawatlas-emergency-declarations.csv'
# SDG-4-Quality-Education
url_school = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-4-Quality-Education/covid-19-school-closure.txt'
url_remote = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SDG-4-Quality-Education/covid-19_remote_learning.xlsx?raw=true'
url_duration = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SDG-4-Quality-Education/covid-9_duration_school_closures.csv'

# SDG-8-Work-Economic-Grow
url_child = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SDG-8-Work-and-Economic-Growth/child-labor.xlsx?raw=true'
url_finm = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SDG-8-Work-and-Economic-Growth/covid-19_financial_measures.xlsx?raw=true'
url_women = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-8-Work-and-Economic-Growth/proportion_women_managerial.csv'
url_response = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-8-Work-and-Economic-Growth/response_countries.csv'
url_unemployment = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SDG-8-Work-and-Economic-Growth/unemployment_rate_2021.xlsx?raw=true'
url_working = 'https://raw.githubusercontent.com/ArtemioPadilla/UNYouthHackaton2021/main/Datasets/SDG-8-Work-and-Economic-Growth/working_hours_lost_covid.csv'
url_youthunemployment = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SDG-8-Work-and-Economic-Growth/youth_unemployed.xlsx?raw=true'

# SGD-1-No-Poverty
url_household = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SGD-1-No-Poverty/covid-19_households.xlsx?raw=true'
url_social = 'https://github.com/ArtemioPadilla/UNYouthHackaton2021/blob/main/Datasets/SGD-1-No-Poverty/social_protection.xlsx?raw=true'

In [ ]:
cases = pd.read_csv(url_cases)
us_mandates = pd.read_csv(url_US)
responses = pd.read_csv(url_response, index_col=0)

In [ ]:
def findCountry (country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_2
    except:
        return ("not founded")

responses['code'] = responses.apply(lambda row: findCountry(row.Country) , axis = 1)
responses.head()

,Country,Response_measure,date_start,date_end,code
_id,,,,,
1,Austria,AdaptationOfWorkplace,2020-03-16T00:00:00,4/13/2020,AT
2,Austria,AdaptationOfWorkplace,2021-04-01T00:00:00,NaN,AT
3,Austria,BanOnAllEvents,2020-11-03T00:00:00,5/18/2021,AT
4,Austria,ClosDaycare,2020-03-16T00:00:00,5/4/2020,AT
5,Austria,ClosDaycare,2020-11-17T00:00:00,12/7/2020,AT


In [ ]:
cases = cases[['Date_reported', 'Country_code', 'WHO_region', 'New_cases','New_deaths']]

In [ ]:
def cols_datetime(cols, df):
  for i in cols:
    df[i] = pd.to_datetime(df[i])
  
def creating_ma(code):
  dfcode = cases[cases.Country_code == code]
  cols = ['New_cases','New_deaths']
  for i in cols:
    #dfcode['MA3_'+i] = dfcode[i].rolling(window=3).mean()
    dfcode['MA7_'+i] = dfcode[i].rolling(window=7).mean()
    dfcode['MA21_'+i] = dfcode[i].rolling(window=21).mean()
  return dfcode

cases_real = pd.concat([creating_ma(code) for code in list(cases.Country_code.unique())])

cols_datetime(['date_start','date_end'], responses)
#creating_ma(['New_cases', 'New_deaths'],cases)

In [ ]:
def get_plots(code, measure):
  #columns = ['New_cases','Cumulative_cases', 'New_deaths', 'Cumulative_deaths']
  columns = ['New_cases', 'New_deaths',]
  country = pycountry.countries.get(alpha_2= code).name
  dfcode = cases_real[cases_real.Country_code == code]
  df = responses[responses.code == code]
  if measure in df.Response_measure.unique():
    for i in columns:
      dmax= dfcode[i].values.max()
      dmin= dfcode[i].values.min()
      stra = " ".join(i.split('_'))
      fig = go.Figure()
      #fig.add_trace(go.Scatter(name= stra,x=dfcode['Date_reported'] , y=dfcode[i]))
      fig.update_layout(
          title=country + ' ' + stra + ' with measure: ' + measure,
          xaxis_title="Date",
          yaxis_title= stra,
      )
      if 'New' in i:
        #fig.add_trace(go.Scatter(name="MA3",mode="lines",x=dfcode["Date_reported"],  y=dfcode["MA3_"+i],))
        fig.add_trace(go.Scatter(name="MA7",mode="lines",x=dfcode["Date_reported"],  y=dfcode["MA7_"+i],))
        #fig.add_trace(go.Scatter(name="MA21",mode="lines",x=dfcode["Date_reported"],  y=dfcode["MA21_"+i],))
      
      df1 = df[df.Response_measure == measure][['date_start','date_end']].reset_index(drop=True)
      m = len(df1)
      for j in range(m):
        xa = df1.iloc[j]['date_start']
        ya = df1.iloc[j]['date_end']
        if not pd.isna(xa):
          fig.add_trace(go.Scatter(x=[xa,xa], y=[dmin,dmax], mode='lines', line=dict(color='orange', width=2, dash='dash'), name='Start '+xa.strftime("%d %b, %Y") ))
        if not pd.isna(ya):
          fig.add_trace(go.Scatter(x=[ya,ya], y=[dmin,dmax], mode='lines', line=dict(color='black', width=1, dash='dot'), name='End '+ya.strftime("%d %b, %Y") ))
      fig.show()

In [ ]:
def uni_measures(code):
  df = responses[responses.code == code]
  return list(df.Response_measure.unique())

In [ ]:
responses.code.unique()

array(['AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE',
       'GR', 'HU', 'IS', 'IE', 'IT', 'LV', 'LI', 'LT', 'LU', 'MT', 'NL',
       'NO', 'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE', 'CH', 'GB'],
      dtype=object)

In [ ]:
def plots(code, cols):
  for i in cols:
    get_plots(code, i)
cols_responses = ['StayHomeOrder', 'StayHomeGen', 'StayHomeRiskG', 'SocialCircle', 'PrivateGatheringRestrictions', 'ClosPubAny', 'MasksVoluntaryAllSpaces', 'MasksMandatoryAllSpaces', 'WorkplaceClosures', 'Teleworking', 'AdaptationOfWorkplace']


plots('ES', cols_responses)

---

```'StayHomeOrder', 'StayHomeGen', 'StayHomeRiskG', 'SocialCircle', 'PrivateGatheringRestrictions', 'ClosPubAny', 'MasksVoluntaryAllSpaces', 'MasksMandatoryAllSpaces', 'WorkplaceClosures', 'Teleworking', 'AdaptationOfWorkplace'```

### LSTM Series

In [ ]:
BLUE_COLOR = "#37689e"
TURQUOISE_COLOR = "#9a21e4"
LIGHT_BLUE_COLOR = "#9bbdcf"
YELLOW_COLOR = "#e49a21"
LIGHT_YELLOW_COLOR = "#f2c576"
RED_COLOR = "#da2f20"
LIGHT_RED_COLOR = "#e66f4b"
Blue = '#21cde4'
tf.random.set_seed(221)
def plot_prediction(code, col):
  

  strcol = " ".join(col.split('_'))
  country = pycountry.countries.get(alpha_2= code).name

  trn = cases_real[cases_real.Country_code == code ][['Date_reported',col]].reset_index(drop=True)
  trn['Date_reported'] = pd.to_datetime(trn['Date_reported'])
  cases_data = trn[col].values
  cases_data = cases_data.reshape((-1,1))

  split_percent = 0.80
  split = int(split_percent*len(cases_data))

  cases_train = cases_data[:split]
  cases_test = cases_data[split:]

  date_train = trn['Date_reported'][:split]
  date_test = trn['Date_reported'][split:]
  look_back = 15

  train_generator = TimeseriesGenerator(cases_train, cases_train, length=look_back, batch_size=20)     
  test_generator = TimeseriesGenerator(cases_test, cases_test, length=look_back, batch_size=1)
  model = Sequential()
  model.add(
      LSTM(10,
          activation='relu',
          input_shape=(look_back,1))
  )
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')

  num_epochs = 25
  model.fit_generator(train_generator, epochs=num_epochs, verbose=0)
  prediction = model.predict_generator(test_generator)

  cases_train = cases_train.reshape((-1))
  cases_test = cases_test.reshape((-1))
  prediction = prediction.reshape((-1))

  trace1 = go.Scatter(
      x = date_train,
      y = cases_train,
      mode = 'lines',
      name = 'Data'
  )
  trace2 = go.Scatter(
      x = date_test,
      y = prediction,
      mode = 'lines',
      name = 'Prediction'
  )
  trace3 = go.Scatter(
      x = date_test,
      y = cases_test,
      mode='lines',
      name = 'Ground Truth'
  )
  layout = go.Layout(
      title = country + " "  + strcol +" with Prediction/Testing",
      xaxis = {'title' : "Date"},
      yaxis = {'title' : strcol }
  )
  fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
  #fig.show()

  cases_data = cases_data.reshape((-1))
  def predict(num_prediction, model):
      prediction_list = cases_data[-look_back:]
      
      for _ in range(num_prediction):
          x = prediction_list[-look_back:]
          x = x.reshape((1, look_back, 1))
          out = model.predict(x)[0][0]
          prediction_list = np.append(prediction_list, out)
      prediction_list = prediction_list[look_back-1:] # - look_back
          
      return prediction_list

  def predict_dates(num_prediction):
      last_date = trn['Date_reported'].values[-1]
      prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
      return prediction_dates

  num_prediction = 60
  forecast = predict(num_prediction, model)
  forecast_dates = predict_dates(num_prediction)
  trace1 = go.Scatter(
      x = trn['Date_reported'],
      y = trn[col],
      mode = 'lines',
      name = 'Cases',
      #marker_color = BLUE_COLOR,
      marker_color = Blue,
  )

  trace2 = go.Scatter(
      x = forecast_dates,
      y = forecast,
      mode = 'lines',
      name = 'Prediction'
  )

  layout = go.Layout(
      title={
        'text': country + ": "+ strcol + " prediction in the next 60 days" ,
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
      xaxis = {'title' : "Date" },
      yaxis = {'title' : strcol },
      legend=dict(
        x=0.85,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    ))

  fig = go.Figure(data=[trace1, trace2], layout=layout)
  fig.show()
  keras.backend.clear_session()

**Low-Income Countries**

In [ ]:
plot_prediction('ET', 'New_cases')

In [ ]:
plot_prediction('ET', 'New_deaths')

In [ ]:
plot_prediction('KH', 'New_cases')

In [ ]:
plot_prediction('KH', 'New_deaths')

**High-income countries** based on CLustering Analysis

In [ ]:
plot_prediction('IL', 'New_cases')

In [ ]:
plot_prediction('IL', 'New_deaths')

In [ ]:
plot_prediction('DE', 'New_cases')

In [ ]:
plot_prediction('DE', 'New_deaths')